In [1]:
import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading


seed = 46

world = Village()
village_side = 39
village_height = 10
world.build(village_height=village_height, village_side=village_side, building_every=5, density=0.15, seed=seed)
#vgraph_builder = world.to_drake_plant()
print(len(world.obstacles))

/home/peter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


INFO:drake:Meshcat listening for connections at http://localhost:7001
INFO:drake:Allocating contexts to support 20 parallel queries given omp_num_threads 20 omp_max_threads 20 and omp_thread_limit 2147483647 OpenMP enabled in build? true


475


In [3]:
import pickle
from pydrake.all import HPolyhedron, VPolytope


regions = []
with open('experiment_village_greedy_39_10_09_27_04_59_24___46_500_0.050_0.200/data/it_6.pkl', 'rb') as f:
    data= pickle.load(f)

for rga, rgb in zip(data['ra'], data['rb']):
    for ra, rb in zip(rga, rgb):
        regions.append(HPolyhedron(ra, rb))

# Plotting Utils

In [4]:
import os
import mcubes
from meshcat.geometry import TriangularMeshGeometry, MeshLambertMaterial
from matplotlib.colors import to_hex as _to_hex

to_hex = lambda rgb: '0x' + _to_hex(rgb)[1:]
def col_hand(q1,q2,q3):
    p =np.array([q1,q2,q3])
    # if np.any(world.L_dom>p):
    #     return 1
    # if np.any(world.U_dom<p):
    #     return 1
    return 1.0*world.col_handle(np.array([q1,q2,q3]))

def col_hand_neg(q1,q2,q3):
    p =np.array([q1,q2,q3])
    if np.any(world.L_dom>p):
        return 0
    if np.any(world.U_dom<p):
        return 0
    return 1- 1.0*world.col_handle(p)

def plot_collision_constraint(N = 50, q_min = np.array(world.L_dom)-5, q_max= np.array(world.U_dom)+5):
    if f"col_cons{N}.pkl" in os.listdir('tmp'):
        with open(f"tmp/col_cons{N}.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(
                q_min), tuple(
                q_max), N, N, N, col_hand, 0.5)
        with open(f"tmp/col_cons{N}.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)

    #tri_drake = [SurfaceTriangle(*t) for t in triangles]

    #vertices += _offset_meshcat_2.reshape(-1,3)
    obj = world['col']
    color = to_hex((1,0,0))
    material = MeshLambertMaterial(color=color, opacity=0.4,  wireframe=False)
    obj.set_object(TriangularMeshGeometry(vertices, triangles),
                                   material)

def plot_collision_constraint_neg(N = 50, q_min = np.array(world.L_dom)-5, q_max= np.array(world.U_dom)+5):
    if f"col_cons{N}_n.pkl" in os.listdir('tmp'):
        with open(f"tmp/col_cons{N}_n.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(
                q_min), tuple(
                q_max), N, N, N, col_hand_neg, 0.5)
        with open(f"tmp/col_cons{N}_n.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)

    #tri_drake = [SurfaceTriangle(*t) for t in triangles]

    #vertices += _offset_meshcat_2.reshape(-1,3)
    obj = world['col_n']
    color = to_hex((0,1,0))
    material = MeshLambertMaterial(color=color, opacity=0.3,  wireframe=False)
    obj.set_object(TriangularMeshGeometry(vertices, triangles),
                                   material)
from meshcat.geometry import Cylinder
import meshcat.transformations as tf
from meshcat.geometry import MeshLambertMaterial

def compute_rotation_matrix(a, b):
    # Normalize the points
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    
    # Calculate the rotation axis
    rotation_axis = np.cross(a, b)
    rotation_axis /= np.linalg.norm(rotation_axis)
    
    # Calculate the rotation angle
    dot_product = np.dot(a, b)
    rotation_angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    
    # Construct the rotation matrix using Rodrigues' formula
    skew_matrix = np.array([[0, -rotation_axis[2], rotation_axis[1]],
                            [rotation_axis[2], 0, -rotation_axis[0]],
                            [-rotation_axis[1], rotation_axis[0], 0]])
    rotation_matrix = np.eye(3) + np.sin(rotation_angle) * skew_matrix + (1 - np.cos(rotation_angle)) * np.dot(skew_matrix, skew_matrix)
    
    return rotation_matrix

def plot_edge(pt1, pt2, name, color, opacity, size = 0.01):
    
    material = MeshLambertMaterial(color=color, opacity=opacity)
    world[name].set_object(
                        Cylinder(np.linalg.norm(pt1-pt2), size),
                        material)
    
    dir = pt2-pt1
    rot = compute_rotation_matrix(np.array([0,1,0]), dir )
    #print(rot)
    offs = rot@np.array([0,np.linalg.norm(pt1-pt2)/2,0])
    tfmat = np.zeros((4,4))
    tfmat[:3,:3] = rot
    tfmat[:3,-1] = offs + pt1.squeeze()
    tfmat[3,3] = 1
    world[name].set_transform(tfmat)
    
def plot_edges(edges, name, color, opacity  = 1, size = 0.01):
    for i, e in enumerate(edges):
         plot_edge(e[0], e[1], name + f"/e_{i}", color, opacity, size=size)

#plot_edge(np.zeros(3), 5*np.ones(3), 'test', (1,0,0), 1, 0.8)

In [5]:
world.plot_regions(regions, opacity=0.6)
from utils import plot_regions
 #this is for debugging the meldis meshcat -> activate kproximity to see the world
plot_regions(world.meshcat_drake, regions)

# Dummy Pathplanning and plotting of the resulting trajectory

In [6]:
from dijkstraspp import DijkstraSPPsolver

def conv_dummy(q):
    return q

dspp = DijkstraSPPsolver(regions, conv_dummy)

[DijkstraSPP] Pre-Building adjacency matrix  0 / 83
[DijkstraSPP] Pre-Building adjacency matrix  10 / 83
[DijkstraSPP] Pre-Building adjacency matrix  20 / 83
[DijkstraSPP] Pre-Building adjacency matrix  30 / 83
[DijkstraSPP] Pre-Building adjacency matrix  40 / 83
[DijkstraSPP] Pre-Building adjacency matrix  50 / 83
[DijkstraSPP] Pre-Building adjacency matrix  60 / 83
[DijkstraSPP] Pre-Building adjacency matrix  70 / 83
[DijkstraSPP] Pre-Building adjacency matrix  80 / 83
[DijkstraSPP] Pre-Building d-adjacency matrix  0 / 293


In [7]:
point_in_regions(np.array([0,0, 0.5]), regions)
point_in_regions(np.array([38,38, 6.5]), regions)


True

In [8]:
wps, dist = dspp.solve(np.array([0,0, 0.5]), np.array([38,38, 6.5]), refine_path=True)


[DijkstraSPP] optimized distance/ start-distance = 60.22 / 69.35 = 0.87


In [9]:
edges = []
for idx, wp in enumerate(wps[:-1]):
    edges.append([wp, wps[idx+1]])

plot_edges(edges, 'traj', (1,1,0), 0.5, 0.1 )

In [11]:
from meshcat.transformations import translation_matrix, rotation_matrix
sign_size = [20.4, 20.2]  # Width and height of the sign
sign_position = [-20.0, 0.0, 10.0]  # X, Y, Z position of the sign
sign_color = [0.7, 0.7, 0.7]  # RGB color for the sign
from meshcat.geometry import MeshLambertMaterial, ImageTexture, Box
from PIL import Image
import meshcat
# Add the sign as a rectangle

# Load an image to put on the sign
image_path = "village_sign.png"

# Create a textured material for the sign
texture = ImageTexture(image = meshcat.geometry.PngImage.from_file(image_path))
textured_material = MeshLambertMaterial(map=texture)
sign = Box((sign_size[0], sign_size[1], 0.1))
world["sign"].set_object(sign, textured_material)
transl = translation_matrix(sign_position)
rot = rotation_matrix(np.pi/2, [1,0,0])
rot = rot@rotation_matrix(np.pi/2, [0,1,0])
rot[:,3] = transl[:,3]
world["sign"].set_transform(rot)



In [ ]:
# this saves the result to a html which you can share
# file = world.static_html()
# with open('Village.html', 'w') as f:
#     f.write(file)

In [9]:
# if f"col_cons{100}_n.pkl" in os.listdir('tmp'):
#     with open(f"tmp/col_cons{100}_n.pkl", 'rb') as f:
#         d = pickle.load(f)
#         vertices = d['vertices']
#         triangles = d['triangles']

# output_file_path = "output_mesh.obj"

# # Write the mesh data to the .obj file
# with open(output_file_path, 'w') as obj_file:
#     obj_file.write("# Triangle Mesh\n")
    
#     # Write vertex coordinates
#     for vertex in vertices:
#         vertex = vertex
#         obj_file.write(f"v {' '.join(map(str, vertex))}\n")
    
#     # Write triangle definitions
#     for triangle in triangles:
#         # Add 1 to vertex indices because .obj files use 1-based indexing
#         obj_file.write(f"f {' '.join(map(lambda x: str(x + 1), triangle))}\n")

# print(f"Mesh saved to {output_file_path}")